# Homolog BinaryMap Encoding

## Import `Python` modules

In [1]:
import pandas as pd
import re
import json
import binarymap as bmap
import matplotlib.pyplot as plt
import seaborn as sns
import jax
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
from jax.experimental import sparse
from jaxopt import ProximalGradient
import jaxopt
import numpy as onp
from scipy.stats import pearsonr

from timeit import default_timer as timer
%matplotlib inline

In [2]:
import sys
sys.path.append("..")
from multidms.utils import create_homolog_modeling_data

## Strategy for converting mutations to be relative to the reference homolog

As described above, our strategy involves converting all mutations in all homologs to be relative to the amino-acid sequence of the reference homolog. The below function performs this step.

Note 1: we will probably need some special-purpose code to handle gaps. This isn't done yet.

Next, we will test the above function with a small test case. Below, we define variants from two imaginary homologs: "reference" and "2".

In [3]:
homologs = {
    "reference" : "MG",
    "2" : "MP"
}

In [4]:
test_dict = {
    'homolog' : ["reference","reference","reference","2","2","2","2","2"],
    'variant' : ['M1E', 'G2R', 'G2P', 'M1E', 'P2R', 'P2G', 'M1E P2G', 'M1E P2R'],
    'log2E' : [2, -7, -0.5, 2.3, -5, 0.4, 2.7, -2.7],
}
test_df = pd.DataFrame(test_dict)
test_df

,homolog,variant,log2E
0,reference,M1E,2.0
1,reference,G2R,-7.0
2,reference,G2P,-0.5
3,2,M1E,2.3
4,2,P2R,-5.0
5,2,P2G,0.4
6,2,M1E P2G,2.7
7,2,M1E P2R,-2.7


In [5]:
(X, y), updated_func_score_df, all_subs = create_homolog_modeling_data(test_df, homologs, "homolog", "variant", "log2E")
updated_func_score_df

,homolog,variant,log2E,var_wrt_ref
0,reference,M1E,2.0,M1E
1,reference,G2R,-7.0,G2R
2,reference,G2P,-0.5,G2P
3,2,M1E,2.3,M1E G2P
4,2,P2R,-5.0,G2R
5,2,P2G,0.4,
6,2,M1E P2G,2.7,M1E
7,2,M1E P2R,-2.7,M1E G2R
